In [2]:
!pip install keras==2.15.0

In [3]:
import keras
keras.__version__

2024-04-11 10:25:32.981605: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 10:25:32.981722: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 10:25:33.095968: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


'2.15.0'

In [4]:
classes = {0 : 'CNV',
           1 : 'DME',
           2 : 'Drusen',
           3 : 'Normal'}

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout
from keras.models import Sequential
from keras.layers import Dense,Input,Flatten
from tensorflow.keras.utils import load_img,img_to_array
from sklearn.metrics import confusion_matrix

In [6]:
import os
os.listdir('/kaggle/input/kermany2018/OCT2017 /train')

['DRUSEN', 'CNV', 'NORMAL', 'DME']

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_data = train_datagen.flow_from_directory('/kaggle/input/kermany2018/OCT2017 /train',
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset='training')
test_data = train_datagen.flow_from_directory('/kaggle/input/kermany2018/OCT2017 /train',
                                              target_size=(224,224),
                                              batch_size=32,
                                              class_mode='categorical',
                                              shuffle=False,
                                              subset='validation')

Found 75138 images belonging to 4 classes.
Found 8346 images belonging to 4 classes.


In [8]:
train_data.image_shape

(224, 224, 3)

In [9]:
np.unique(test_data.labels)

array([0, 1, 2, 3], dtype=int32)

In [10]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler, ReduceLROnPlateau
from keras.optimizers import SGD
from keras.regularizers import l2
import keras.backend as K
import math

In [11]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))

x = base_model.output
x = AveragePooling2D()(x)
x = Dropout(.5)(x)
x = Flatten()(x)
x = Dense(4, kernel_initializer='glorot_uniform', kernel_regularizer=l2(.0005), activation='softmax')(x)
model2 = Model(inputs=base_model.input, outputs=x)
model2.summary()

87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     

In [12]:
opt = SGD(lr=.1, momentum=.9)
model2.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping
results2 = model2.fit(train_data, epochs=10, validation_data=test_data,
                  steps_per_epoch=len(train_data), validation_steps=len(test_data),
                  callbacks = EarlyStopping(patience=5, monitor='val_accuracy',restore_best_weights=True)
                  )

Epoch 1/10


2024-04-11 10:29:32.662897: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1712831378.557132     106 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2349/2349 [==============================] - 999s 414ms/step - loss: 0.2306 - accuracy: 0.9268 - val_loss: 0.2237 - val_accuracy: 0.9342
Epoch 2/10
2349/2349 [==============================] - 421s 179ms/step - loss: 0.1185 - accuracy: 0.9625 - val_loss: 0.2158 - val_accuracy: 0.9412
Epoch 3/10
2349/2349 [==============================] - 421s 179ms/step - loss: 0.1150 - accuracy: 0.9639 - val_loss: 0.2555 - val_accuracy: 0.9285
Epoch 4/10
2349/2349 [==============================] - 418s 178ms/step - loss: 0.0791 - accuracy: 0.9750 - val_loss: 0.1673 - val_accuracy: 0.9468
Epoch 5/10
2349/2349 [==============================] - 418s 178ms/step - loss: 0.0911 - accuracy: 0.9714 - val_loss: 0.1657 - val_accuracy: 0.9485
Epoch 6/10
2349/2349 [==============================] - 417s 178ms/step - loss: 0.0620 - accuracy: 0.9797 - val_loss: 0.2087 - val_accuracy: 0.9448
Epoch 7/10
2349/2349 [==============================] - 417s 177ms/step - loss: 0.0495 - accuracy: 0.9836 - val_loss: 0.194

In [14]:
loss2, acc2 = model2.evaluate(test_data)

261/261 [==============================] - 35s 133ms/step - loss: 0.2088 - accuracy: 0.9427


In [15]:
print("Inception v3")
print("Test Accuracy:",round(acc2*100,2),"%","\nTest Loss:",loss2)

Inception v3
Test Accuracy: 94.27 % 
Test Loss: 0.20878268778324127


In [16]:
model2.save('oct_model_Inception.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [17]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink(r'oct_model_Inception.h5')

/kaggle/working


/kaggle/working/oct_model_Inception.h5